In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta

In [8]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
team_names = [team['full_name'] for team in nba_teams]
team_names.sort()
team_ids = [team['id'] for team in nba_teams]

In [34]:
games = pd.read_pickle('../data/pkl/sbr_current_betting_data_2023-03-09.pkl')

In [74]:
df=pd.DataFrame()

In [72]:
games[['Betmgm_Spread', 'Betmgm_Odds']]

,Betmgm_Spread,Betmgm_Odds
0,-3.5,-110
1,+3.5,-110
2,+4.5,-110
3,-4.5,-110
4,+9.5,-110
5,-9.5,-110
6,-2.5,-110
7,+2.5,-110
8,+9.5,-110
9,-9.5,-110


In [53]:
checkboxes = [
        'Betmgm',
        'Draft_Kings',
        'Fanduel',
        'Caesars',
        'Pointsbet',
        'Wynn',
        'Betrivers'
    ]

In [41]:
import streamlit as st

In [47]:
checkbox_values = {}
for label in checkboxes:
    checkbox_values[label] = st.checkbox(label, value=True)

In [ ]:
def display_dataframe(checkbox_values, df):
        # Hide the columns that are not selected
        columns_to_drop = [label if not checkbox_values[label]]
        df_filtered = df.drop(columns_to_drop, axis=1)

        # Display the data frame with the selected columns using streamlit
        st.write(df_filtered)

In [64]:
checkbox_values = {
        'Betmgm': 'Betmgm',
        'Draft_Kings': 'Draft_Kings',
        'Fanduel': 'Fanduel',
        'Caesars': 'Caesars',
        'Pointsbet': 'Pointsbet',
        'Wynn': 'Wynn',
        'Betrivers': 'Betrivers'
    }

In [63]:
[label for label in checkboxes if not checkbox_values[label]]

['Betrivers']

# Basic boxscores

In [27]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import scoreboard

# Get yesterday's date
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(days=1)
yesterday_str = yesterday.strftime('%m/%d/%Y')

scoreboard_ = scoreboard.Scoreboard(game_date=yesterday_str, league_id='00', day_offset=0)
games = scoreboard_.game_header.get_data_frame()
if not games.empty:
    game_ids = list(games['GAME_ID'])
    print(team['full_name'], game_ids)

# Advanced boxscores

The command above/below gets the advanced boxscore from a given game id

boxscores_advanced.get_data_frames()[0] returns the advanced boxscore broken up per player

boxscores_advanced.get_data_frames()[1] returns the advanced boxscore broken up per team

In [25]:
from nba_api.stats.endpoints import boxscoreadvancedv2

boxscores_advanced_player = None
boxscores_advanced_team = None

for game_id in game_ids:
    if boxscores_advanced_team is None:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = gamefinder.get_data_frames()[1]
        boxscores_advanced_player = gamefinder.get_data_frames()[0]
    else:
        gamefinder = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscores_advanced_team = pd.concat([boxscores_advanced_team, gamefinder.get_data_frames()[1]])
        boxscores_advanced_player = pd.concat([boxscores_advanced_player, gamefinder.get_data_frames()[0]])

CPU times: user 155 ms, sys: 23.6 ms, total: 179 ms
Wall time: 2.69 s


In [29]:
boxscores_advanced_team.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE',
       'PACE_PER40', 'POSS', 'PIE'],
      dtype='object')

In [77]:
# boxscores_advanced_team_yesterday.to_pickle('boxscores_advanced_team_part2.pkl')

In [78]:
# boxscores_advanced_player_yesterday.to_pickle('boxscores_advanced_player_part2.pkl')

In [30]:
df = pd.read_pickle('../data/pkl/raw_games_5yrs.pkl')

In [32]:
df['GAME_DATE']

0     2023-03-04
1     2023-03-03
2     2023-02-28
3     2023-02-26
4     2023-02-24
         ...    
426   2018-03-15
427   2018-03-13
428   2018-03-10
429   2018-03-08
430   2018-03-06
Name: GAME_DATE, Length: 13700, dtype: datetime64[ns]